In [1]:
with open('doc.txt',"r",encoding="utf-8") as f:
    raw_data=f.read()

In [4]:
import tiktoken
tokenizer=tiktoken.get_encoding("gpt2")

In [7]:
enc_text=tokenizer.encode(raw_data)
print(len(enc_text))

1570


In [ ]:
tokenizer.decode()

'🌙 The Moon-Sealed Quest — A Story of Arcana, Friendship, and Destiny\nIn a land where forests breathed secrets and rivers carried echoes of forgotten gods, there lay a kingdom called Atherwyn. It shimmered beneath two moons—one silver, one violet—whose lights intertwined to weave the magic that flowed through every tree, stone, and living soul.\nBut one night, the violet moon flickered… and vanished.\nA chill swept through Atherwyn. Magic dimmed like dying embers. Spells faltered. The wind whispered a name it had not spoken in a thousand years:\n“Eclipsara.”\nChapter 1 — The Girl Who Heard the Moon Cry\nFar from Atherwyn’s marble cities, in the misty village of Lyr’s End, lived a girl named Seren Vale. She was seventeen, with dusk-dark hair and eyes that sometimes shimmered like moonlight—especially when she dreamed.\nAnd that night, she dreamed of a sky bleeding purple.\nShe woke to a strange ache in her chest. Stepping outside, Seren saw the silver moon alone in the heavens, trembli

In [11]:
context_length=4
x=enc_text[:context_length]
y=enc_text[context_length]
print(f"{x}--->{y}")

[8582, 234, 247, 383]--->6869


In [21]:
for i in range (1,context_length+1):
    context=enc_text[:i]
    desired=enc_text[i]
    print(tokenizer.decode(context),"--->",tokenizer.decode([desired]))
   



� ---> �
� ---> �
🌙 --->  The
🌙 The --->  Moon


In [45]:
import torch
from torch.utils.data import Dataset

class GPTdatasetv1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunks = token_ids[i : i + max_length]
            target_chunks = token_ids[i + 1 : i + max_length + 1]

            self.input_ids.append(torch.tensor(input_chunks, dtype=torch.long))
            self.target_ids.append(torch.tensor(target_chunks, dtype=torch.long))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    
 

In [53]:
from torch.utils.data import DataLoader
import tiktoken

def create_dataloader_v1(
        txt,
        batch_size=4,
        max_length=256,
        stride=128,
        shuffle=True,
        drop_last=True,
        num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTdatasetv1(txt, tokenizer, max_length, stride)

    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

In [58]:
import torch
dataloader=create_dataloader_v1(raw_data,batch_size=8,max_length=4,stride=1,shuffle=False)
data_iter=iter (dataloader)
inputs,target=next(data_iter)
print("INPUTS\n",inputs)
print("targets\n",target)



INPUTS
 tensor([[8582,  234,  247,  383],
        [ 234,  247,  383, 6869],
        [ 247,  383, 6869,   12],
        [ 383, 6869,   12,   50],
        [6869,   12,   50, 2287],
        [  12,   50, 2287,  276],
        [  50, 2287,  276, 6785],
        [2287,  276, 6785,  851]])
targets
 tensor([[ 234,  247,  383, 6869],
        [ 247,  383, 6869,   12],
        [ 383, 6869,   12,   50],
        [6869,   12,   50, 2287],
        [  12,   50, 2287,  276],
        [  50, 2287,  276, 6785],
        [2287,  276, 6785,  851],
        [ 276, 6785,  851,  317]])
